In [ ]:
#default_exp models

In [ ]:
#export
from mantisshrimp.imports import *

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Models" data-toc-modified-id="Models-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Models</a></span></li><li><span><a href="#Export" data-toc-modified-id="Export-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Export</a></span></li></ul></div>

# Models
> Models for object detection

In [ ]:
#export
# TODO: How to properly inject coco_evaluator? A callback/metric?
class RCNNModel(LightningModule):
    def __init__(self, n_class):
        super().__init__()
        self.m = self.create_model(n_class)
        
    def create_model(self, n_class, h=256): raise NotImplementedError
        
    def forward(self, x): return self.m(x)
    
    def training_step(self, b, b_idx):
        x,y = b
        losses = self.m(x,list(y))
        loss = sum(losses.values())
        return {'loss': loss, 'log': {'avg_loss': loss, **losses}}
    
    def validation_step(self, b, b_idx):
        xb,yb = b
        with torch.no_grad(): preds = self(xb)
        preds = [{k:v.to(torch.device('cpu')) for k,v in p.items()} for p in preds]
        res = {y["image_id"].item():pred for y,pred in zip(yb, preds)}
        self.coco_evaluator.update(res)
        
    def validation_epoch_end(self, outs):
        self.coco_evaluator.synchronize_between_processes()
        self.coco_evaluator.accumulate()
        self.coco_evaluator.summarize()
        return {}
        
    
    def configure_optimizers(self):
        params = [p for p in self.parameters() if p.requires_grad]
        opt = torch.optim.SGD(params, 5e-3, momentum=0.9, weight_decay=0.0005)
        step_lr = torch.optim.lr_scheduler.StepLR(opt, step_size=3, gamma=0.1)
        return [opt], [step_lr]

In [ ]:
#export
class MaskRCNNModel(RCNNModel):
    def create_model(self, n_class, h=256):
        model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
        in_features = model.roi_heads.box_predictor.cls_score.in_features
        model.roi_heads.box_predictor = FastRCNNPredictor(in_features, n_class)
        in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
        model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask, h, n_class)
        return model

In [ ]:
#export
class FastRCNNModel(RCNNModel):
    def create_model(self, n_class, h=256):
        model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
        in_features = model.roi_heads.box_predictor.cls_score.in_features
        model.roi_heads.box_predictor = FastRCNNPredictor(in_features, n_class)
        return model

# Export

In [ ]:
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 02_data.core.ipynb.
Converted 04_data.annotations.ipynb.
Converted 06_transforms.ipynb.
Converted 07_data.load.ipynb.
Converted 08_models.ipynb.
Converted 11_evaluation.coco.ipynb.
Converted Untitled.ipynb.
Converted Untitled1.ipynb.
Converted index.ipynb.
